In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM

dataset = load_dataset("uonlp/CulturaX", "lt")

model_name = "meta-llama/Llama-2-7b"
# model_name = "meta-llama/Llama-2-7b-hf"

README.md:   0%|          | 0.00/32.6k [00:00<?, ?B/s]

lt_part_00000.parquet:   0%|          | 0.00/3.11G [00:00<?, ?B/s]

lt_part_00001.parquet:   0%|          | 0.00/3.11G [00:00<?, ?B/s]

lt_part_00002.parquet:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

lt_part_00003.parquet:   0%|          | 0.00/3.11G [00:00<?, ?B/s]

...

lt_part_00009.parquet:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

lt_part_00010.parquet:   0%|          | 0.00/2.45G [00:00<?, ?B/s]

lt_part_00011.parquet:   0%|          | 0.00/1.70G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/112 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType

model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(model_name, load_in_4bit=True, torch_dtype=torch.float16, device_map="auto")

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, peft_config)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=4000,
    num_proc=12,
    remove_columns=["text"],
    load_from_cache_file=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map (num_proc=12):   0%|          | 0/13339785 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("./tokenized_culturax")

Saving the dataset (0/72 shards):   0%|          | 0/13339785 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!du -sh tokenized_culturax

34G	tokenized_culturax


In [ ]:
!zip -r /content/drive/MyDrive/tokenized_culturax.zip tokenized_culturax

  adding: tokenized_culturax/ (stored 0%)
  adding: tokenized_culturax/dataset_dict.json (stored 0%)
  adding: tokenized_culturax/train/ (stored 0%)
  adding: tokenized_culturax/train/data-00038-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00056-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00062-of-00072.arrow (deflated 78%)
  adding: tokenized_culturax/train/data-00026-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00058-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00015-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/state.json (deflated 92%)
  adding: tokenized_culturax/train/data-00055-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00042-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00049-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/train/data-00020-of-00072.arrow (deflated 75%)
  adding: tokenized_culturax/t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/tokenized_culturax.zip -d /content/

Archive:  /content/drive/MyDrive/tokenized_culturax.zip
   creating: /content/tokenized_culturax/
 extracting: /content/tokenized_culturax/dataset_dict.json  
   creating: /content/tokenized_culturax/train/
  inflating: /content/tokenized_culturax/train/data-00038-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00056-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00062-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00026-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00058-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00015-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/state.json  
  inflating: /content/tokenized_culturax/train/data-00055-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00042-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00049-of-00072.arrow  
  inflating: /content/tokenized_culturax/train/data-00020-

In [ ]:
from datasets import load_from_disk

tokenized_dataset = load_from_disk("file:///content/tokenized_culturax")

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
import torch

model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = LlamaForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model = get_peft_model(model, peft_config)


In [ ]:
target_tokens = 100_000_000
tokens_per_sample = 512
num_examples = target_tokens // tokens_per_sample  # ~195,000

tokenized_dataset = tokenized_dataset['train'].select(range(num_examples))

training_args = TrainingArguments(
    output_dir="./llama2-lt-culturax",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_dir="./logs",
    save_total_limit=2,
    logging_steps=50,
    save_steps=500,
    bf16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

trainer.save_model("./llama2-lt-culturax-final")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,2.867800
100,2.794200
150,2.760200
200,2.697700
250,2.692500
300,2.660300
350,2.635900
400,2.635700
450,2.629300
500,2.592700


In [ ]:
!unzip /content/tuned-model.zip -d /content/

Archive:  /content/tuned-model.zip
  inflating: /content/tuned-model/adapter_config.json  
  inflating: /content/tuned-model/adapter_model.safetensors  
  inflating: /content/tuned-model/optimizer.pt  
  inflating: /content/tuned-model/README.md  
  inflating: /content/tuned-model/rng_state.pth  
  inflating: /content/tuned-model/scheduler.pt  
  inflating: /content/tuned-model/special_tokens_map.json  
  inflating: /content/tuned-model/tokenizer.model  
  inflating: /content/tuned-model/tokenizer_config.json  
  inflating: /content/tuned-model/trainer_state.json  
  inflating: /content/tuned-model/training_args.bin  


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/tuned-model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, b

In [ ]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset

dataset = load_dataset("neurotechnology/lithuanian-qa-v1", split="train")

def tokenize(batch):
    prompts = [f"Question: {q}\nAnswer:" for q in batch['question']]
    answers = batch['answer']

    full_texts = [p + " " + a for p, a in zip(prompts, answers)]
    tokenized = tokenizer(full_texts, truncation=True, padding="max_length", max_length=512)

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/13848 [00:00<?, ? examples/s]

In [ ]:
import torch
import math
from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(b["input_ids"]) for b in batch])
    attention_mask = torch.stack([torch.tensor(b["attention_mask"]) for b in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": input_ids}

dataloader = DataLoader(tokenized_dataset, batch_size=8, collate_fn=collate_fn)

In [ ]:
total_loss = 0.0
total_tokens = 0

model.cuda()
with torch.no_grad():
    for batch in dataloader:
        batch = {k: v.cuda() for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item() * batch["input_ids"].numel()
        total_tokens += batch["input_ids"].numel()

avg_loss = total_loss / total_tokens
perplexity = math.exp(avg_loss)
print(f"Perplexity: {perplexity:.2f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 156.12 MiB is free. Process 171053 has 14.59 GiB memory in use. Of the allocated memory 14.49 GiB is allocated by PyTorch, and 1.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)